# PyGeom: Gemc Python Geometry Tutorial

This file is more a tutorial. To get the very first steps, please look at the index.ipny file in this directory.
In this file I assume you have ROOT installed and have the PyGeom package available. 

# Table of Contents

1. [Getting Started](#getting_started)
1. [The Geomtery Engine](#the_geometry_engine) <br>
    1.1 [Importing TXT tables](#importing_txt_tables) <br>
    1.2 [Inspecting the Geometry](#inspecting_the_geometry) <br>    
1. [Geometry Objects](#geometry_objects) <br>
    1.1 [Drawing the Geometry](#drawing_the_geometry) <br>
    1.2 [Modifying Geometry](#modifying_geometry) <br>
    1.3 [Directly interacting with the ROOT objects](#Directly_interacting_with_the_ROOT_objects)

<a id="getting_started"></a>
# Getting Started

To the the ROOT viewing functionality, you need to have installed ROOT on your system and have sourced the "thisroot.sh" or "thisroot.csh" file so that the ROOT libraries can be found.

Similarly, to use the MySQL database connectivity feature, you need to have "MySQLdb" installed on your system. This is usually done with: "pip install mysql-python" (python 2), or "pip install mysqlclient" (python 3). 

Usually you will not have the PyGeom package installed as a sub-directory of the directory you are working in. You can do two things to make PyGeom available anywhere you are working:
1. Add the directory to your PYTHONPATH. In bash: `export PYTHONPATH=/my/directory/PyGeom:${PYTHONPATH}` or in tcsh: `setenv PYTHONPATH /my/directory/PyGeom:$PYTHONPATH`
2. In your Python script, add the following, where you want to replace the '/data/CLAS12/gemc/PyGeom' with the directory where you placed the PyGeom files:

In [1]:
import sys
sys.path.append('/data/CLAS12/gemc/PyGeom')

Where '/data/CLAS12/gemc/PyGeom' is the path where I happen to have installed the PyGeom package.

_Future versions of PyGeom will come with a setup.py file so you have the option to install the package in a systems location._

You should now be able to import the PyGeom package. 

In [2]:
import PyGeom

Welcome to JupyROOT 6.09/03


<a id="the_geometry_engine"></a>
# The GeometryEngine


The class that stores all individual items for your geometry is the `GeometryEngine`. From the Python docs:

In [3]:
help(PyGeom.GeometryEngine)

Help on class GeometryEngine in module PyGeom.GeometryEngine:

class GeometryEngine
 |  A class for building GEANT4 geometries.
 |  The initial purpose is to build the geometries for the MySQL database or text files used by Gemc1 or Gemc2.
 |  Each GeometryEngine object represents ONE 'detector' (as defined by Gemc).
 |  Expansion to other geometry formats is possible, see for instance the GeometryROOT class.
 |  
 |  Class initialization parameters:
 |  
 |  gen = PyGeom.GeometryEngine(
 |      detector = "MyName"    # String. Name of your detector. This will determine the name of text output files.
 |      variation= "original"  # String. The variation for your detector. Defaults to 'original'
 |      table_id = 0           # Int.    The table identity in the MySQL database. Default = 0.
 |      db_host  = "my.db.org" # String. Name of the database host computer.
 |                             # If specified, attempt to connect to the DB. Default = 0 - do not connect.
 |      user = 

Most of the time you would only need to specify the detector name when initializing this class. The database paramters can be specified at initialization, or later when connecting to a database with the GeometryEngine.MySQL_OpenDB() function.

A typical command to start the GeometryEngine would thus be:

In [4]:
gen = PyGeom.GeometryEngine("MyDetector")

You can now add Geometry() objects into this GeometryEngine(), or, as shown below, you can load a text table (or MySQL table) into the geometry.

<a id="importing_txt_tables"></a>
## Importing TXT Tables

One nice feature of the GeometryEngine is that it can not only write out GEMC tables, but also read them in. It can do the same with MySQL tables, which can be read as well as written. You can thus use the GeometryEngine to convert between different file formats. One such format for output is Python code, which can be added to a script.

Here is an example: read the geometry for the beamline from the standard CLAS12 detector, then show the geometry in a ROOT window and write out a Python template file. Again, you would want to replace the long string '/data/CLAS12/...' with the location of the TXT file you are interested in rendering. Note that TGeoManager is a bit verbose (pink text)

In [5]:
gen.TXT_Read_Geometry("/data/CLAS12/gemc/detectors/clas12/beamline/beamline__geometry_FTOn.txt")
rr = PyGeom.GeometryROOT()
rr.build_volumes(gen)
rr.close_geometry()
rr.draw("OGLX")            # Draw the geometry on the screen. On a Mac use "ogl" instead of "OGLX"
gen.Python_Write("test")  # Write the file Template_MyDetector.py

Info in <TGeoManager::TGeoManager>: Geometry GEMC, Python ROOT Geometry Engine for GEMC created
Info in <TGeoManager::SetTopVolume>: Top volume is root. Master volume is root
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Info in <TGeoManager::CheckGeometry>: Fixing runtime shapes...
Info in <TGeoManager::CheckGeometry>: ...Nothing to fix
Info in <TGeoManager::CloseGeometry>: Counting nodes...
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CloseGeometry>: Building cache...
Info in <TGeoManager::CountLevels>: max level = 2, max placements = 10
Info in <TGeoManager::CloseGeometry>: 12 nodes/ 12 volume UID's in Python ROOT Geometry Engine for GEMC
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


The picture shows the rendering of the beamline. In a Jupyter notebook (of this file), you can rotate and zoom the picture. If you did this on your own computer in Python, you would get a ROOT window in which you can move and manipulate the picure. 

The final command wrote a new file called "Template_MyDetector.py", which contains a python function "calculate_MyDetector_geometry(g_en)", that re-creates the geometry in a geometry engine object that is passed to it. You can then edit this file, rename the function if desired, and change parameters of the geometric objects. You can read it all back in by importing the file into Python and calling the function.

<a id="inspecting_the_geometry"></a>
## Inspecting the Geometry
If you want to inspect the Geometry object you just created, just print it:

In [6]:
print gen

Database:  NOT OPENED 
Table   : MyDetector
Geometry: 
     aluminumBeamPipe in root ::aluminum beampipe
     vacuumBeamPipe in aluminumBeamPipe ::vacuum inside aluminum beampipe
     airPipe in root ::air line from target to vacuum line
     tungstenConeFT in root ::FT tungsten moller shield
     mountToTorus in root ::mount to torus
     mountToMount in root ::beamline mount to torus mount
     mountInnerShielding in root ::beamline mount to torus beamline inner shield
     mountOuterShielding in root ::beamline mount to torus beamline outer shield
     tungstenTorusBeamShield in root ::tungsten beampipe shield inside torus
     downstreamShield in root ::Shielding pipe after the torus
     downstreamNose in root ::Shielding nose after the torus



You can retrieve individual Geomtery objects by name or number. The Geometry class is both a "list" and a "dictionary" in the Python sense. or You can also search for items, which can useful with large geometries.

In [7]:
print gen[3]  # Print the *fourth* item in the list
print "-----------------------------"
print gen['mountToTorus'] # Print the item by name

tungstenConeFT | root | FT tungsten moller shield | 0.0*mm 0.0*mm 877.4*mm  | 0.0*cm 0.0*cm 0.0*cm  | ff0000 | Polycone | 0.0*deg 360.0*deg 4.0*counts 35.1028*mm 35.1028*mm 40.1*mm 40.1*mm 38.1508*mm 71.0925912531*mm 71.0925912531*mm 76.0476*mm 0.0*mm 767.2*mm 767.2*mm 882.6*mm  | beamline_W | no | 1 | 1 | 1 | 1 | 1 | no | no | no 
-----------------------------
mountToTorus | root | mount to torus | 0.0*mm 0.0*mm 2358.7*mm  | 0.0*cm 0.0*cm 0.0*cm  | 55ff55 | Polycone | 0.0*deg 360.0*deg 4.0*counts 76.2*mm 76.2*mm 76.2*mm 76.2*mm 95.25*mm 95.25*mm 126.9*mm 126.9*mm 0.0*mm 382.3*mm 382.3*mm 395.0*mm  | G4_STAINLESS-STEEL | no | 1 | 1 | 1 | 1 | 1 | no | no | no 


You could also have used gen.find_volume('mountToTorus')

If you want to search without a complete name, you can use find_volume_regex, which will return a list of found volumes. Eg:

In [8]:
mount_vols = gen.find_volume_regex('mount.*')
print "The number of volumes with mount is: ",len(mount_vols)
print "---------------------"
for vol in mount_vols:
    print vol.name

The number of volumes with mount is:  4
---------------------
mountToTorus
mountToMount
mountInnerShielding
mountOuterShielding


<a id="geometry_objects"></a>
# Geometry Objects

You can create new primitive geometry objects using the Geomtery class. Each such geometry object is stored in the GeometryEngine, and these objects can be combined using boolean geometry operators: "add", "subtract".

In [9]:
help(PyGeom.Geometry)

Help on class Geometry in module PyGeom.Geometry:

class Geometry
 |  A Class for storing the geometry of an objects to be rendered in the output.
 |  You would typicaly add these objects into a GeometryEngine object.
 |  Each of the paramters corresponds to the parameters defined in the GEMC text
 |  input format. For great detail, check the GEMC documentation.
 |  
 |  gobj = Geometry(
 |       name="unknown",  # String. Name of the object.
 |       mother="root",   # String. Name of the mother of this object.
 |       description="",  # String. Text description of the object.
 |       pos=[0,0,0],     # List.   Position of the object in a list of 3 floats.
 |       pos_units="cm",  # String. Units for the position of the object: mm, cm, inch, m
 |       rot=[0,0,0],     # List.   Angles defining the rotation of the object.
 |       rot_units="rad", # String. Units for the angles: deg, rad, mrad
 |       rot_order="",    # String. Order of the angles: "xyz", "zyx" etc.
 |       col="

In [10]:
gen2 = PyGeom.GeometryEngine("PaddleTest")
paddle = PyGeom.Geometry(name="paddle",mother="root",description="Example of a paddle",
                  pos=[0, 0, 20], pos_units='cm', 
                  rot=[0, 0, 0], rot_units='rad',
                  col='339999', 
                  g4type='Box', 
                  dimensions=[2,2,10],dims_units='cm',
                  material='ScintillatorB')

**Some notes:**
   - If you forget what parameters are needed/allowed, type: help(Geometry) at the prompt, and look at the documentation which shows you all the options.
   - There is a bit of flexibility build into the Geometry. You can specify the position as a "vector": [x,y,z], or you can specify it as a GEMC style string: " 10.*cm 15.2*cm 1000*mm ".
   - More flexibility, if you want to mix units, then specify them as an array. So <code> pos="10.*cm 15.2*cm 1000*mm" </code> is the same as <code> pos=[10.,15.2,1000.],pos_units=['cm','cm','mm'] </code>.
   - See the section "Unit Conversions" for information on converting units automatically.
   - It is MUCH safer to specify a name for every argument, because then the order is not important. If you are careful about the order, and don't skip arguments, you can leave out the names (like C++ or Fortran).  

In [11]:
print paddle

paddle | root | Example of a paddle | 0*cm 0*cm 20*cm  | 0*rad 0*rad 0*rad  | 339999 | Box | 2*cm 2*cm 10*cm  | ScintillatorB | no | 1 | 1 | 1 | 1 | 1 | no |  |  


Now we add this geometry object to the GeometryEngine: 

In [12]:
gen2.add(paddle)

Let'd also add a target, at (0,0,0), made of liquid hydrogen (G4_lH2), and small flat pancake shape: a tube of 2 cm radius and 2 mm thickness. To show you a different way of entering the volume, we do the whole thing in one line:

In [13]:
gen2.add( PyGeom.Geometry(name="target",mother="root",description="A thin LH2 target",
                  pos="0*cm 0*cm 0*cm",
                  rot="0*deg 0*deg 0*deg",
                  g4type="Tube",
                  dimensions=" 0*cm 2*cm 2*mm 0*deg 360*deg",
                  material="G4_lH2",
                  col="CC0000"))

After you have added a number of items, you may now want to inspect what you have in your geometry. See more in [Inspecting the Geometry](#inspecting_the_geometry)

In [14]:
print gen2[1]

target | root | A thin LH2 target | 0.0*cm 0.0*cm 0.0*cm  | 0.0*deg 0.0*deg 0.0*deg  | CC0000 | Tube | 0.0*cm 2.0*cm 2.0*mm 0.0*deg 360.0*deg  | G4_lH2 | no | 1 | 1 | 1 | 1 | 1 | no |  |  


You can draw the geometry. See below.

<a id="drawing_the_geometry"></a>
## Drawing the geometry

To render the geometry you created with ROOT, you first of all need to have the ROOT environment setup, using the standard method. Often all this takes is to source "thisroot.sh" in your shell, *before* you start ipython, or jupyter notebook.



In [15]:
rr2=PyGeom.GeometryROOT(gen2)

Warning in <TGeoManager::Init>: Deleting previous geometry: GEMC/Python ROOT Geometry Engine for GEMC
Info in <TGeoManager::TGeoManager>: Geometry PaddleTest, Geometry for PaddleTest variation: original created
Info in <TGeoManager::SetTopVolume>: Top volume is root. Master volume is root


Note that some matials are not defined yet in the GeometryROOT package. I am working on adding these, please be patient. For showing the volumes, and for GEMC, these missing materials do not matter at all, they would only be important if you want to track particles through your ROOT geometry. Doing that would still need a significant amount of code...

You can now draw the geomerty by calling the "draw()" method of your GeometryROOT object.
If you do this is a *live* notebook (either a root --notebook, or jupyter notebook, or on binder), then the picture produced below will rotate, zoom etc, just as it would if you executed these command in a python shell.

In [16]:
rr2.draw("oglx")    
# Note: the 'option' "olgx", draws the picture with OpenGL for this notebook. For Python, you may need "OGL" instead.
# If you leave the option out, the default will be used, which may be a wireframe picture.

<a id="modifying_geometry"></a>
## Modifying Geometry

Since this is a interactive session for geometry, you probably want to be able to adjust one of your objects, and then look at it again. Let's shorten the paddle from 10 cm to 5 cm.

We need to manipulate the object inside the GeometryEngine object, "gen". What we do is change the length, and then look at it again.
**Note:** We can only have one active GeometryROOT object at one time right now. So we re-initialize our rr object to clear out the old geometry and load the new one. 

In [17]:
gen2['paddle'].dimensions[2]=5

In [18]:
rr3 = PyGeom.GeometryROOT(gen2)

Warning in <TGeoManager::Init>: Deleting previous geometry: PaddleTest/Geometry for PaddleTest variation: original
Info in <TGeoManager::TGeoManager>: Geometry PaddleTest, Geometry for PaddleTest variation: original created
Info in <TGeoManager::SetTopVolume>: Top volume is root. Master volume is root


In [19]:
rr3.draw("oglx")

<a id="Directly_interacting_with_the_ROOT_objects"></a>
## Directly interacting with the ROOT objects
It is possible to directly interact with the underlying ROOT objects. This is more *expert level*, but I just want to show you some fun. Let's change the geometry of the paddle again, but now only in ROOT.
We now need to access the volume in the store of ROOT objects, in rr._volumes, which is a dictionary.

In [20]:
import numpy as np
ndims=np.array([10,10,1],dtype='double')
rr3._volumes['paddle'].GetShape().SetDimensions(ndims)
rr3.draw("oglx")

If you are interested in details of the object, you can ask ROOT about it. The Dump() method for ROOT object gives you the information on that object.

In [21]:
rr3._volumes['paddle'].Dump()

==> Dumping object at: 0x00007ff090579920, name=paddle, class=TGeoVolume

*fNodes                       ->0                 array of nodes inside this volume
*fShape                       ->7ff09058c7b0      shape
*fMedium                      ->7ff08c9622d0      tracking medium
*fFinder                      ->0                 finder object for divisions
*fVoxels                      ->0                 finder object for bounding boxes
*fGeoManager                  ->7ff08d83e800      ! pointer to TGeoManager owning this volume
*fField                       ->0                 ! just a hook for now
fOption                                           ! option - if any
fOption.fRep                  ->7ff0905799d8      ! String data
fOption.fRep.                 ->7ff0905799d8      
fNumber                       2                   volume serial number in the list of volumes
fNtotal                       0                   total number of physical nodes
fRefCount                     1    

Note that since ROOT is C++, the Dump() gives you C++ style informatio. Interacting with the ROOT objects requires proper type casting, which is why you needed to create a numpy array of doubles for the changed geometry.